# Homework 1 - Berkeley STAT 157

Handout 1/22/2017, due 1/29/2017 by 4pm in Git by committing to your repository. Please ensure that you add the TA Git account to your repository.

1. Write all code in the notebook.
1. Write all text in the notebook. You can use MathJax to insert math or generic Markdown to insert figures (it's unlikely you'll need the latter). 
1. **Execute** the notebook and **save** the results.
1. To be safe, print the notebook as PDF and add it to the repository, too. Your repository should contain two files: ``homework1.ipynb`` and ``homework1.pdf``. 

The TA will return the corrected and annotated homework back to you via Git (please give `rythei` access to your repository).

In [30]:
from mxnet import ndarray as nd
from tqdm import tqdm
import time

## 1. Speedtest for vectorization

Your goal is to measure the speed of linear algebra operations for different levels of vectorization. You need to use `wait_to_read()` on the output to ensure that the result is computed completely, since NDArray uses asynchronous computation. Please see http://beta.mxnet.io/api/ndarray/_autogen/mxnet.ndarray.NDArray.wait_to_read.html for details. 

1. Construct two matrices $A$ and $B$ with Gaussian random entries of size $4096 \times 4096$. 
1. Compute $C = A B$ using matrix-matrix operations and report the time. 
1. Compute $C = A B$, treating $A$ as a matrix but computing the result for each column of $B$ one at a time. Report the time.
1. Compute $C = A B$, treating $A$ and $B$ as collections of vectors. Report the time.
1. Bonus question - what changes if you execute this on a GPU?

In [7]:
A = nd.random.normal(shape = (4096,4096))
B = nd.random.normal(shape = (4096,4096))

In [17]:
nd.array([1,2])


[1. 2.]
<NDArray 2 @cpu(0)>

In [25]:
import time 
from tqdm import tqdm
tic = time.time()
C1 = nd.dot(A,B)
print(time.time()-tic)
C1.wait_to_read()
print("OK")
tic = time.time()
C2 = nd.zeros((A.shape[0], B.shape[1]))
for i in tqdm(range(B.shape[1])):
    C2[:,i] = nd.dot(A,B[:,i])
print(time.time()-tic)
C2.wait_to_read()
print("OK2")
tic = time.time()
C3 = nd.zeros((A.shape[0], B.shape[1]))
for i in tqdm(range(A.shape[0])):
    for j in range(B.shape[1]):
        C3[i, j] = nd.dot(A[i], B[:,j])
print(time.time()-tic)
C3.wait_to_read()

0.0068378448486328125


  6%|▌         | 248/4096 [00:00<00:01, 2416.09it/s]

OK


100%|██████████| 4096/4096 [00:01<00:00, 2475.20it/s]


1.6634180545806885


  0%|          | 0/4096 [00:00<?, ?it/s]

OK2


100%|██████████| 4096/4096 [1:08:20<00:00,  1.12it/s]

4100.584766864777


In [33]:
A = nd.random.normal(shape = (4096,4096), ctx = mx.gpu())
B = nd.random.normal(shape = (4096,4096), ctx = mx.gpu())
import time 
from tqdm import tqdm
tic = time.time()
C1 = nd.dot(A,B)
print(time.time()-tic)
C1.wait_to_read()
print("OK")
tic = time.time()
C2 = nd.zeros((A.shape[0], B.shape[1]))
for i in tqdm(range(B.shape[1])):
    C2[:,i] = nd.dot(A,B[:,i])
print(time.time()-tic)
C2.wait_to_read()
print("OK2")
tic = time.time()
C3 = nd.zeros((A.shape[0], B.shape[1]))
for i in tqdm(range(A.shape[0])):
    for j in range(B.shape[1]):
        C3[i, j] = nd.dot(A[i], B[:,j])
print(time.time()-tic)
C3.wait_to_read()

  0%|          | 0/4096 [00:00<?, ?it/s]

0.00012350082397460938
OK


100%|██████████| 4096/4096 [00:01<00:00, 3279.09it/s]


1.2518465518951416


  0%|          | 0/4096 [00:00<?, ?it/s]

OK2


  1%|          | 21/4096 [00:28<1:33:00,  1.37s/it]

KeyboardInterrupt: 

The matrix-matrix calculation speed up by quite a bit (factor of 60?) using a GPU. The rest is about the same.

## 2. Semidefinite Matrices

Assume that $A \in \mathbb{R}^{m \times n}$ is an arbitrary matrix and that $D \in \mathbb{R}^{n \times n}$ is a diagonal matrix with nonnegative entries. 

1. Prove that $B = A D A^\top$ is a positive semidefinite matrix. 
1. When would it be useful to work with $B$ and when is it better to use $A$ and $D$?

1. For $B$ to be positive semidefinite, we would need it to be symmetric and $x^TBx = x^TADA^Tx \geq 0$ for nonzero $x\in \mathbb{R}^{m}$. $B$ is clearly symmetrix since $B^T = AD^TA^T = ADA^T = B$ since $D^T = D$ due to it being diagonal. For the second part, note that we can rewrite $x^TBx$ as 
\[ x^TADA^Tx = (x^TAD^{1/2})(D^{1/2}A^Tx) = ||D^{1/2}A^Tx||^2 \geq 0,\]
as desired. 

2. Working with B is useful 

## 3. MXNet on GPUs

1. Install GPU drivers (if needed)
1. Install MXNet on a GPU instance
1. Display `!nvidia-smi`
1. Create a $2 \times 2$ matrix on the GPU and print it. See http://d2l.ai/chapter_deep-learning-computation/use-gpu.html for details.

In [3]:
!nvidia-smi

Tue Jan 29 07:02:45 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.79       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           On   | 00000000:00:1E.0 Off |                    0 |
| N/A   48C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [5]:
import mxnet as mx
x = nd.random.normal(shape=(2,2), ctx = mx.gpu())
print(x)


[[-1.3204551   0.68232244]
 [-0.9858383   0.01992839]]
<NDArray 2x2 @gpu(0)>


## 4. NDArray and NumPy 

Your goal is to measure the speed penalty between MXNet Gluon and Python when converting data between both. We are going to do this as follows:

1. Create two Gaussian random matrices $A, B$ of size $4096 \times 4096$ in NDArray. 
1. Compute a vector $\mathbf{c} \in \mathbb{R}^{4096}$ where $c_i = \|A B_{i\cdot}\|^2$ where $\mathbf{c}$ is a **NumPy** vector.

To see the difference in speed due to Python perform the following two experiments and measure the time:

1. Compute $\|A B_{i\cdot}\|^2$ one at a time and assign its outcome to $\mathbf{c}_i$ directly.
1. Use an intermediate storage vector $\mathbf{d}$ in NDArray for assignments and copy to NumPy at the end.

In [7]:
import numpy as np 

In [8]:
A = nd.random.normal(shape=(4096,4096))
B = nd.random.normal(shape=(4096,4096))
c = np.zeros(4096)

In [31]:
tic = time.time()
for i in tqdm(range(B.shape[1])):
    c[i] = nd.norm(nd.dot(A, B[:,i])).asscalar()**2
print(time.time()-tic)

100%|██████████| 4096/4096 [00:49<00:00, 82.60it/s]

49.59265398979187


In [32]:
d = nd.zeros(4096)
tic = time.time()
for i in tqdm(range(B.shape[1])):
    d[i] = nd.norm(nd.dot(A, B[:,i])).asscalar()**2
c = d.asnumpy()
print(time.time()-tic)

100%|██████████| 4096/4096 [00:50<00:00, 81.78it/s]

50.09121012687683


## 5. Memory efficient computation

We want to compute $C \leftarrow A \cdot B + C$, where $A, B$ and $C$ are all matrices. Implement this in the most memory efficient manner. Pay attention to the following two things:

1. Do not allocate new memory for the new value of $C$.
1. Do not allocate new memory for intermediate results if possible.

In [ ]:
D = C.zeros_like()
C += 

## 6. Broadcast Operations

In order to perform polynomial fitting we want to compute a design matrix $A$ with 

$$A_{ij} = x_i^j$$

Our goal is to implement this **without a single for loop** entirely using vectorization and broadcast. Here $1 \leq j \leq 20$ and $x = \{-10, -9.9, \ldots 10\}$. Implement code that generates such a matrix.

In [39]:
x = nd.arange(-10.0, 10.1, 0.1).reshape((201,1))
y = nd.arange(1, 21, 1).reshape((1,20))
A = x**y 
print(A)


[[-1.0000000e+01  1.0000000e+02 -1.0000000e+03 ...  9.9999998e+17
  -1.0000000e+19  1.0000000e+20]
 [-9.8999996e+00  9.8009995e+01 -9.7029895e+02 ...  8.3451338e+17
  -8.2616820e+18  8.1790647e+19]
 [-9.8000002e+00  9.6040001e+01 -9.4119208e+02 ...  6.9513558e+17
  -6.8123289e+18  6.6760824e+19]
 ...
 [ 9.8000011e+00  9.6040024e+01  9.4119232e+02 ...  6.9513681e+17
   6.8123409e+18  6.6760952e+19]
 [ 9.8999996e+00  9.8009995e+01  9.7029895e+02 ...  8.3451338e+17
   8.2616820e+18  8.1790647e+19]
 [ 1.0000000e+01  1.0000000e+02  1.0000000e+03 ...  9.9999998e+17
   1.0000000e+19  1.0000000e+20]]
<NDArray 201x20 @cpu(0)>
